In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.metrics import classification_report
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import multiprocessing

In [7]:
df_data = pd.read_csv('Data/cleaned_hotel_reviews.csv')
df_data

,Review,Rating,label,clean_punc,lemmatized_review,number_review
0,nice hotel expensive parking got good deal sta...,4,positive,nice hotel expensive parking got good deal sta...,nice hotel expensive park get good deal stay h...,nice hotel expensive park get good deal stay h...
1,ok nothing special charge diamond member hilto...,2,negative,ok nothing special charge diamond member hilto...,ok nothing special charge diamond member hilto...,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,3,neutral,nice rooms not 4 experience hotel monaco seat...,nice room not 4 experience hotel monaco seattl...,nice room not experience hotel monaco seattle ...
3,"unique, great stay, wonderful time hotel monac...",5,positive,unique great stay wonderful time hotel monac...,unique great stay wonderful time hotel monaco ...,unique great stay wonderful time hotel monaco ...
4,"great stay great stay, went seahawk game aweso...",5,positive,great stay great stay went seahawk game aweso...,great stay great stay go seahawk game awesome ...,great stay great stay go seahawk game awesome ...
...,...,...,...,...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5,positive,best kept secret 3rd time staying charm not 5...,best keep secret 3rd time stay charm not 5 sta...,best keep secret time stay charm not star ca n...
20487,great location price view hotel great quick pl...,4,positive,great location price view hotel great quick pl...,great location price view hotel great quick pl...,great location price view hotel great quick pl...
20488,"ok just looks nice modern outside, desk staff ...",2,negative,ok just looks nice modern outside desk staff ...,ok just look nice modern outside desk staff n ...,ok just look nice modern outside desk staff n ...
20489,hotel theft ruined vacation hotel opened sept ...,1,negative,hotel theft ruined vacation hotel opened sept ...,hotel theft ruin vacation hotel open sept 17 2...,hotel theft ruin vacation hotel open sept gues...


In [12]:
encoding_label = {
    1:0,
    2:1,
    3:2,
    4:3,
    5:4
}

In [13]:
df_data['Rating'] = df_data['Rating'].map(encoding_label)
df_data = pd.get_dummies(df_data, columns=['Rating'])
df_data

,Review,label,clean_punc,lemmatized_review,number_review,Rating_0,Rating_1,Rating_2,Rating_3,Rating_4
0,nice hotel expensive parking got good deal sta...,positive,nice hotel expensive parking got good deal sta...,nice hotel expensive park get good deal stay h...,nice hotel expensive park get good deal stay h...,False,False,False,True,False
1,ok nothing special charge diamond member hilto...,negative,ok nothing special charge diamond member hilto...,ok nothing special charge diamond member hilto...,ok nothing special charge diamond member hilto...,False,True,False,False,False
2,nice rooms not 4* experience hotel monaco seat...,neutral,nice rooms not 4 experience hotel monaco seat...,nice room not 4 experience hotel monaco seattl...,nice room not experience hotel monaco seattle ...,False,False,True,False,False
3,"unique, great stay, wonderful time hotel monac...",positive,unique great stay wonderful time hotel monac...,unique great stay wonderful time hotel monaco ...,unique great stay wonderful time hotel monaco ...,False,False,False,False,True
4,"great stay great stay, went seahawk game aweso...",positive,great stay great stay went seahawk game aweso...,great stay great stay go seahawk game awesome ...,great stay great stay go seahawk game awesome ...,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",positive,best kept secret 3rd time staying charm not 5...,best keep secret 3rd time stay charm not 5 sta...,best keep secret time stay charm not star ca n...,False,False,False,False,True
20487,great location price view hotel great quick pl...,positive,great location price view hotel great quick pl...,great location price view hotel great quick pl...,great location price view hotel great quick pl...,False,False,False,True,False
20488,"ok just looks nice modern outside, desk staff ...",negative,ok just looks nice modern outside desk staff ...,ok just look nice modern outside desk staff n ...,ok just look nice modern outside desk staff n ...,False,True,False,False,False
20489,hotel theft ruined vacation hotel opened sept ...,negative,hotel theft ruined vacation hotel opened sept ...,hotel theft ruin vacation hotel open sept 17 2...,hotel theft ruin vacation hotel open sept gues...,True,False,False,False,False


In [14]:
x_train, x_test, y_train, y_test = train_test_split(df_data['lemmatized_review'], df_data.iloc[:,5:], test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16392,), (4099,), (16392, 5), (4099, 5))

In [15]:
cores = multiprocessing.cpu_count()

w2v = Word2Vec(
    window=5,
    vector_size=100,
    sample=6e-5, 
    alpha=0.03, 
    min_alpha=0.0007, 
    negative=20,
    workers=cores-1
)

In [16]:
w2v.build_vocab(x_train, progress_per=10000)

In [17]:
w2v.train(x_train, total_examples=w2v.corpus_count, epochs=30, report_delay=1)

(13673888, 334047000)

In [18]:
w2v.init_sims(replace=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_8588\2512947105.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


In [21]:
similar_words = w2v.wv.most_similar(positive=["rooms"])
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

KeyError: "Key 'rooms' not present in vocabulary"